In [22]:
import pandas as pd
from collections import Counter
from collections import defaultdict
import matplotlib.pyplot as plt
import sys
import matplotlib

In [23]:
# this approach to set paths is supposed to be better but it's not working.
#from pathlib import Path
#datapath=Path("C:/Users/l_gas/Documents/Development/SpringBoard_DataScience/HealthInsuranceData")
#open2019=datapath / "\\AHRF\\AHRF_2018-2019\\DATA\\AHRF2019.asc"

In [24]:
datapath='C:\\Users\\l_gas\\Documents\\Development\\SpringBoard_DataScience\\HealthInsuranceData'
head2019= datapath + "\\AHRF\\AHRF_2018-2019\\DOC\\AHRF2018-19.sas"

In [25]:
# The source data comes as an ASCII file with fixed column width and no variable names
# Variable names, and their position in the data file can be extracted from the SAS program provided
# Now we open the SAS program clean the data to easily extract any variable of interest from the datafile

# Read the SAS code - create a copy
ahrf19hd=open(head2019)
varlist19=ahrf19hd.readlines()
vardf19=pd.DataFrame(varlist19).copy()


In [26]:
# Keep only the lines with information on how to read each variable in the data file(line starts with @ referencing starting position)
print(vardf19.head(10))
print(vardf19.shape)
locf19=vardf19[vardf19[0].astype(str).str.contains("@")]
print(locf19)
# Keep only the lines with variable lables (the line starts with the variable name and the first letter is always an f)
labf19=vardf19[vardf19[0].astype(str).str[0]=="f"]
print(labf19.shape)

                                                   0
0                                       data ahrf;\n
1            infile 'c:\ahrf2019.asc' lrecl=31706;\n
2                                            input\n
3                                                 \n
4                        /*  AHRF2018-19 SAS FD */\n
5                                                 \n
6       @00001    f00001   $  01.  /*Blank       ...
7       @00002    f00002   $  05.  /*Header - FIP...
8       @00007    f00003   $  05.  /*Entity of Fi...
9       @00012    f00004   $  20.  /*Secondary En...
(14305, 1)
                                                      0
6          @00001    f00001   $  01.  /*Blank       ...
7          @00002    f00002   $  05.  /*Header - FIP...
8          @00007    f00003   $  05.  /*Entity of Fi...
9          @00012    f00004   $  20.  /*Secondary En...
10         @00032    f00005   $  04.  /*Date of File...
...                                                 ...
7148       @31

In [27]:
# Each line in locf19 and labf19 represents one variable
# Clean location information
locf19 = locf19[0].str.split(n=3,expand=True)
locf19.columns=['Pos_Start','Name','Type','Rest']
locf19['String'] = locf19.Type=="$"
locf19['Pos_Start']=locf19['Pos_Start'].map(lambda x: x.lstrip('@'))
locf19['Pos_Start']=locf19['Pos_Start'].astype(int)
locf19 = locf19.reset_index()
locf19 = locf19.drop(['index','Type','Rest'],axis=1)
locend=locf19['Pos_Start'][1:].reset_index().drop(['index'],axis=1)-1
print(locend.info())
locend = locend.rename(columns={"Pos_Start": "Pos_End"})
locf19 = pd.concat([locf19,locend],axis=1)
print(locf19)
print(locf19.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7146 entries, 0 to 7145
Data columns (total 1 columns):
Pos_Start    7146 non-null int32
dtypes: int32(1)
memory usage: 28.0 KB
None
      Pos_Start      Name  String  Pos_End
0             1    f00001    True      1.0
1             2    f00002    True      6.0
2             7    f00003    True     11.0
3            12    f00004    True     31.0
4            32    f00005    True     35.0
...         ...       ...     ...      ...
7142      31690  f1533809   False  31692.0
7143      31693  f1533808   False  31695.0
7144      31696  f1533807   False  31698.0
7145      31699  f1533806   False  31701.0
7146      31702  f0081176   False      NaN

[7147 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7147 entries, 0 to 7146
Data columns (total 4 columns):
Pos_Start    7147 non-null int32
Name         7147 non-null object
String       7147 non-null bool
Pos_End      7146 non-null float64
dtypes: bool(1), float64(1), int32(1

In [28]:
# Clean label information and add to var information
print(labf19.head())
labf19 = labf19[0].str.split("\t",n=1,expand=True)
labf192 =  labf19[1].str.split("\"",n=2,expand=True)
labf19 = pd.concat([labf19[0],labf192[1]],axis=1)
labf19.columns=["Name","Label"]
locf19 = pd.merge(left=locf19,right=labf19, on="Name")
#labf19 = labf19.rename(columns={"0": "Name", "1": "Label"})
print(locf19.head())
print(locf19.info())

                                                     0
7156                           f00001\t \t = "Blank"\n
7157  f00002\t  \t = "Header - FIPS St and Cty Code"\n
7158                   f00003\t\t = "Entity of File"\n
7159         f00004\t\t = "Secondary Entity Of File"\n
7160                     f00005\t\t = "Date of File"\n
   Pos_Start    Name  String  Pos_End                          Label
0          1  f00001    True      1.0                          Blank
1          2  f00002    True      6.0  Header - FIPS St and Cty Code
2          7  f00003    True     11.0                 Entity of File
3         12  f00004    True     31.0       Secondary Entity Of File
4         32  f00005    True     35.0                   Date of File
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7147 entries, 0 to 7146
Data columns (total 5 columns):
Pos_Start    7147 non-null int32
Name         7147 non-null object
String       7147 non-null bool
Pos_End      7146 non-null float64
Label        714

In [29]:
locf19.to_csv('AHRF2019_dict.csv')

In [30]:
locf19.head()

,Pos_Start,Name,String,Pos_End,Label
0,1,f00001,True,1.0,Blank
1,2,f00002,True,6.0,Header - FIPS St and Cty Code
2,7,f00003,True,11.0,Entity of File
3,12,f00004,True,31.0,Secondary Entity Of File
4,32,f00005,True,35.0,Date of File
